In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [2]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51269_2024-04-25~2025-04-25.xlsx')
ka_df = pd.read_excel('/Users/seanyu_mac/Downloads/20250425162628.xlsx')

In [3]:
ka_df.columns

Index(['결제번호', '최초배송비번호', '주문번호', '주문상태', '채널', '발송요청일', '채널상품번호', '상품명', '옵션',
       '수량', '상품금액', '옵션금액', '판매자할인금액', '판매자쿠폰할인금액', '정산기준금액', '기본수수료',
       '노출추가수수료', '추천리워드수수료', '수수료할인금액', '주문일', '송장입력일', '취소완료일', '환불완료일',
       '배송예정일', '판매자상품번호', '배송방법', '택배사', '송장번호', '배송비지불방법', '기본배송비 유형',
       '기본배송비 금액', '도서산간 추가 배송비 금액', '리워드율', '리워드금액', '브랜드', '모델명', '유입경로',
       '옵션코드', '톡딜여부', '상품유형', 'biz판매여부 '],
      dtype='object')

In [4]:
fs_df.columns

Index(['결제 번호', '상품 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '정상가',
       '판매가', '쿠폰 할인', '포인트 할인', '적립금', '정산가', '수령인명', '수령인 연락처', '구매자명',
       '구매자 ID', '구매자 연락처', '주소', '우편번호', '배송 메모', '주문 확인일', '발주 요청일',
       '배송 완료일', '구매 확정일', '배송택배사', '송장 번호', '배송비 정책', '배송비', '도서산간 배송비',
       '취소 완료일', '판매자 취소일', '반품 요청일', '수거 요청일', '수거 완료일', '반품 완료일'],
      dtype='object')

In [5]:
fs_df = fs_df[['결제 번호', '결제 완료일', '주문 상태', '옵션명', '수량', '정산가']]

In [6]:
ka_df = ka_df[['결제번호', '발송요청일', '주문상태', '옵션', '수량', '정산기준금액']]

In [7]:
fs_df['구분'] = '친한스토어'
ka_df['구분'] = '톡스토어'

In [8]:
ka_df['박스수량'] = ka_df['옵션'].apply(lambda x: int(x.split()[1].split('박스')[0]))

In [9]:
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: int(x.split(' (총')[0].split('박스')[0]))

In [10]:
ka_df.columns = fs_df.columns
df = pd.concat([ka_df, fs_df])
df

,결제 번호,결제 완료일,주문 상태,옵션명,수량,정산가,구분,박스수량
0,2429122514,2025-04-25 13:14:56,304 배송 중,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
1,2429108983,2025-04-25 12:56:58,304 배송 중,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
2,2429085249,2025-04-25 12:24:24,304 배송 중,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
3,2429084741,2025-04-25 12:23:41,304 배송 중,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
4,2429040170,2025-04-25 11:25:36,304 배송 중,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
...,...,...,...,...,...,...,...,...
10,parma20250422102510V9gs6,2025-04-22 10:25:42.355000,배송완료,12박스 (총 12개월분)_50%할인,1,260000,친한스토어,12
11,parma202504221025270fWZ7,2025-04-22 10:25:42.029000,배송완료,6박스 (총 6개월분)_45%할인,1,142000,친한스토어,6
12,parma20250422102441W5GJ9,2025-04-22 10:25:26.940000,배송완료,6박스 (총 6개월분)_45%할인,1,142000,친한스토어,6
13,parma20250422102441YYE44,2025-04-22 10:25:10.354000,배송완료,6박스 (총 6개월분)_45%할인,1,142000,친한스토어,6


In [11]:
df['주문 상태'].unique()

array(['304 배송 중', '305 배송 완료', '601 구매 결정', '204 결제 취소 완료', '배송완료',
       '취소완료'], dtype=object)

In [12]:
cancel_df = df[~df['주문 상태'].isin(['304 배송 중', '305 배송 완료', '601 구매 결정', '배송완료'])]
cancel_df

,결제 번호,결제 완료일,주문 상태,옵션명,수량,정산가,구분,박스수량
49,2427048262,2025-04-22 20:22:52,204 결제 취소 완료,선택: 12박스 (총 12개월분)_50%할인,1,260000,톡스토어,12
2,parma20250422105844W9jg0,2025-04-22 10:59:04.306000,취소완료,12박스 (총 12개월분)_50%할인,1,260000,친한스토어,12


In [13]:
df = df[df['주문 상태'].isin(['304 배송 중', '305 배송 완료', '601 구매 결정', '배송완료'])]

In [14]:
df['결제 완료일'] = pd.to_datetime(df['결제 완료일'])
df['결제 완료일'] = df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [15]:
df.rename(columns={'결제 번호': '주문수량'}, inplace=True)
df.rename(columns={'정산가': '매출'}, inplace=True)
result_df = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['매출', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='합계'))
result_df1 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '옵션명'], values='주문수량', aggfunc='count', fill_value=0, margins=True, margins_name='합계'))
result_df['주문수량'] = result_df1['주문수량']
result_df.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     옵션명                                           
2025-04-22 12박스 (총 12개월분)_50%할인         780,000   36    3
           3박스 (총 3개월분)_31%할인           178,000    6    2
           6박스 (총 6개월분)_45%할인         1,278,000   54    9
           선택: 12박스 (총 12개월분)_50%할인   1,300,000   60    5
           선택: 3박스 (총 3개월분)_31%할인     1,780,000   60   20
           선택: 6박스 (총 6개월분)_45%할인     2,556,000  108   18
2025-04-23 선택: 12박스 (총 12개월분)_50%할인     780,000   36    3
           선택: 3박스 (총 3개월분)_31%할인       623,000   21    7
           선택: 6박스 (총 6개월분)_45%할인     1,278,000   54    9
2025-04-24 선택: 3박스 (총 3개월분)_31%할인       267,000    9    3
           선택: 6박스 (총 6개월분)_45%할인       710,000   30    5
2025-04-25 선택: 3박스 (총 3개월분)_31%할인       356,000   12    4
           선택: 6박스 (총 6개월분)_45%할인       710,000   30    5
합계                                   12,596,000  516   93

In [16]:
result_df = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values=['매출', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='합계'))
result_df1 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values='주문수량', aggfunc='count', fill_value=0, margins=True, margins_name='합계'))
result_df['주문수량'] = result_df1['주문수량']
result_df.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     구분                         
2025-04-22 친한스토어   2,236,000   96   14
           톡스토어    5,636,000  228   43
2025-04-23 톡스토어    2,681,000  111   19
2025-04-24 톡스토어      977,000   39    8
2025-04-25 톡스토어    1,066,000   42    9
합계                12,596,000  516   93